In [6]:
import MetaTrader5 as mt5
import pandas as pd
import pytz
from datetime import datetime
from time import gmtime, strftime

def initialize():

    login = 52029003
    password = '0a6Dx9o5!HnsaC'
    server = 'ICMarketsEU-Demo'
    
    # get current date and time
    if not mt5.initialize("C:/Program Files/FTMO MetaTrader 5 - Copy/terminal64.exe"):
        print("initialize() failed, error code =", mt5.last_error())
        quit()
    else:
        print("initialize() success")
        tz_kyjev = pytz.timezone('Europe/Kiev')
        datetime_kyjev = datetime.now(tz_kyjev)
        print("Time: ",datetime_kyjev.strftime("%H:%M:%S"))

    mt5.login(login, password, server)

    acc_info = mt5.account_info()
    if acc_info is None:
        print("Failed to get account info")

    login_number = acc_info.login
    print("Login number: ", login_number)
        # establish connection to the MetaTrader 5 terminal

initialize()

bars = mt5.copy_rates_from_pos("UK100", mt5.TIMEFRAME_M15, 0, 99999)
df = pd.DataFrame(bars)  
df

initialize() success
Time:  18:18:45
Login number:  52029003


,time,open,high,low,close,tick_volume,spread,real_volume
0,1597163400,6158.3,6159.8,6148.8,6156.8,1765,100,0
1,1597164300,6156.8,6159.3,6141.1,6142.4,1465,100,0
2,1597165200,6142.5,6143.8,6123.3,6127.1,1712,100,0
3,1597166100,6127.1,6136.3,6123.6,6127.2,1344,100,0
4,1597167000,6127.3,6130.1,6122.3,6128.8,1101,100,0
...,...,...,...,...,...,...,...,...
99994,1734974100,8073.6,8081.3,8071.8,8075.3,1188,90,0
99995,1734975000,8075.6,8081.3,8071.8,8072.8,1104,90,0
99996,1734975900,8073.3,8074.8,8063.8,8064.3,947,90,0
99997,1734976800,8063.8,8077.1,8063.8,8076.6,742,90,0


In [7]:
def split_by_next_row_three_ways(df):
    """
    Split DataFrame into three based on next row's open/close relationship
    
    Parameters:
        df (pandas.DataFrame): DataFrame with 'open' and 'close' columns
    
    Returns:
        tuple: (price_increased, price_equal, price_decreased)
    """
    # Create conditions for each case
    increased_condition = df['open'] < df['close']
    equal_condition = df['open'] == df['close']
    
    # Shift conditions UP by 1 (looking at next row's conditions)
    next_increased = increased_condition.shift(-1)
    next_equal = equal_condition.shift(-1)
    
    # Fill NaN values with False
    next_increased = next_increased.fillna(False)
    next_equal = next_equal.fillna(False)
    
    # Creating the three separate DataFrames
    price_increased = df[next_increased]
    price_equal = df[next_equal]
    price_decreased = df[~(next_increased | next_equal)]  # Everything else
    
    return price_increased, price_equal, price_decreased

increased, equal, decreased = split_by_next_row_three_ways(df)
increased = increased.copy()
equal = equal.copy()
decreased = decreased.copy()    

print("Rows where next row has open < close:")
print(increased)
print("\nRows where next row has open == close:")
print(equal)
print("\nRows where next row has open > close:")
print(decreased)

Rows where next row has open < close:
             time    open    high     low   close  tick_volume  spread  \
2      1597165200  6142.5  6143.8  6123.3  6127.1         1712     100   
3      1597166100  6127.1  6136.3  6123.6  6127.2         1344     100   
4      1597167000  6127.3  6130.1  6122.3  6128.8         1101     100   
5      1597167900  6128.6  6137.3  6128.6  6134.3         1039     100   
6      1597168800  6134.3  6146.2  6132.3  6143.3          927     100   
...           ...     ...     ...     ...     ...          ...     ...   
99982  1734963300  8093.6  8100.8  8093.6  8097.3          401      90   
99988  1734968700  8084.3  8086.8  8080.3  8081.1          510      90   
99989  1734969600  8081.3  8084.8  8076.6  8084.8          573      90   
99993  1734973200  8085.3  8087.3  8067.8  8073.3         1698      90   
99996  1734975900  8073.3  8074.8  8063.8  8064.3          947      90   

       real_volume  
2                0  
3                0  
4         

In [15]:
increased['open_low_difference'] = increased['open'] - increased['low']
increased['sl'] = increased['high'] - increased['open']

increased[increased['open_low_difference']>= 2].count()


time                   31404
open                   31404
high                   31404
low                    31404
close                  31404
tick_volume            31404
spread                 31404
real_volume            31404
open_low_difference    31404
sl                     31404
dtype: int64

In [18]:
increased[increased['open_low_difference'] < 2].sum()

time                   3.043316e+13
open                   1.355508e+08
high                   1.356528e+08
low                    1.355363e+08
close                  1.356152e+08
tick_volume            6.123604e+06
spread                 4.918823e+06
real_volume            0.000000e+00
open_low_difference    1.449217e+04
sl                     1.019439e+05
dtype: float64